In [1]:
from tqdm import tqdm
import jieba

## FastText文章中使用的搜狗新闻数据集
* Categories
  + sports
  + finance
  + entertainment 
  + automobile
  + technology”

## 下载数据 ，预先处理
1. gbk --> utf-8
2. 从<url>提取类别标签
3. 从<content>提取文本内容

参考资料
* [word2vec 构建中文词向量](http://www.cnblogs.com/Newsteinwell/p/6034747.html)
* [Automatic Online News Issue Construction in Web Environment WWW2008](http://wwwconference.org/www2008/papers/pdf/p457-wang.pdf)

In [3]:
# 这个实验采用搜狗实验室的搜狗新闻语料库，数据链接
# http://www.sogou.com/labs/resource/cs.php
# 1-1. 下载下来的文件名为： news_sohusite_xml.full.zip
# 1-2. 解压 --> xvf news_sohusite_xml.dat
# 1-3.A 字符编码转换  cat news_sohusite_xml.dat | iconv -f gbk -t utf-8 -c | grep "<content>\|<url>"  
# > corpus_labeled.txt
# 1-3.B 使用codec module读入gbk编码的原始文档，使用unicode或者编码为utf-8

file_raw_path = 'data/corpus_labeled.txt'

label_raw = []
data_raw = []
i = 0
with open(file_raw_path, encoding='utf-8') as fr:
    for line in tqdm(fr):
        if i%2==0:
            label_raw.append(line[5:-6])
        else:
            data_raw.append(line[9:-11])
        i += 1


2823992it [00:07, 361426.07it/s]


In [17]:
print('每一个样本有一个url，从中我们可以提取一个话题标签')

[x[:30] for x in label_raw[:len(label_raw):len(label_raw)//10]]


每一个样本有一个url，从中我们可以提取一个话题标签


['http://gongyi.sohu.com/2012070',
 'http://roll.sohu.com/20120625/',
 'http://pic.yule.sohu.com/91258',
 'http://haodf.health.sohu.com/f',
 'http://roll.sohu.com/20120705/',
 'http://db.auto.sohu.com/model_',
 'http://product.it.sohu.com/sea',
 'http://product.it.sohu.com/sea',
 'http://dealer.auto.sohu.com/tj',
 'http://news.sohu.com/20120726/',
 'http://roll.sohu.com/20120706/']

In [23]:
print("统计每个类别的文本数量，对数据有一个初步了解")
labels = []
for label in label_raw:
    labels.append(label[7:].split('.')[0])
from collections import Counter
label_stat = Counter(labels)
for k,v in label_stat.most_common(20):
    print('%15s\t\t%d'%(k,v))

统计每个类别的文本数量，对数据有一个初步了解
           roll		720957
        product		177002
           news		70900
             db		59043
            pic		41236
         sports		38281
          stock		37126
       business		26179
         dealer		25663
            saa		19671
              q		17697
           yule		12779
           drug		11480
          haodf		10890
             it		10797
           data		9110
              s		8678
          money		7448
          daxue		7021
           auto		6843


### 根据论文`Character Level Convolutional Neural Networks for Text Classification (2015)`的描述，选择下述5类话题的样本
1. 'sports'
2. 'stock' // finance
3. 'yule'  // entertainment 
4. 'auto'  // automobile
5. 'it'    // technology”

In [24]:
# 定义lambda函数 去掉文本中怪异符号，参考自
# https://gist.github.com/mrvege/2ba6a437f0a4c4812f21#file-filterpunct-py-L5

punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．＊：；Ｏ？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘-—_…０１２３４５６７８９''')
## 对str/unicode
filterpunt = lambda s: ''.join(filter(lambda x: x not in punct, s))
## 对list
# filterpuntl = lambda l: list(filter(lambda x: x not in punct, l))

In [25]:
cat_selected = set(['sports', 'stock', 'yule', 'auto', 'it'])
label_selected = []
content_selected = []
for i in tqdm(range(len(labels))):
    if labels[i] in cat_selected and len(data_raw[i])>10:
        label_selected.append(labels[i])
        content_selected.append(filterpunt(data_raw[i]))


100%|██████████| 1411996/1411996 [00:09<00:00, 148173.61it/s]


In [26]:
print('corpus样本\n')
for i in range(0, 5000, 1234):
    print('example %d \n\t%s\n\t%s\n' % (i, label_selected[i], content_selected[i]))

corpus样本

example 0 
	stock
	注每次查询最多显示条

example 1234 
	sports
	北京时间月日欧洲杯Ｄ组末轮一场比赛在顿涅茨克顿巴斯竞技场展开争夺英格兰客场比力克乌克兰解禁复出的鲁尼打入全场唯一入球英格兰胜平不败战绩获得小组头名决赛将对阵意大利ｓｐｏｒｔｓ全体育图片社Ｖ行峦月日电　据英国每日邮报报道财大气粗的巴黎圣日耳曼队正密谋以万英镑的价格引进切尔西队长约翰特里巴黎圣日耳曼主帅安切洛蒂曾和特里共事一年日前他本想以万英镑的天价引进ＡＣ米兰后卫蒂亚戈席尔瓦但这桩交易在最后一刻宣布告吹豢ㄋ尔财团注资后的巴黎圣日耳曼队并不在乎花钱他们有望以万英镑的价格引进那不勒斯队射手拉维奇而为了得到特里他们准备承诺一份价值万英镑周薪的天价待遇

example 2468 
	stock
	全景网月日讯　钢铁板块周一早盘走弱ＳＴ广钢跌逾％鲁银投资大金重工等多只个股跌逾％因钢铁行业经营状况继续恶化＞荨毒济参考报记者从中钢协权威人士处获悉纳入钢协统计的大中型钢铁企业中月份盈利情况再度滑至负数其中利润收入为－亿元月度销售利润率为－％全景网／刘磊

example 3702 
	stock
	刘鸿儒开荒修路人苫上拯救股市墒谢挂不要搞　年月深圳和珠海特区成立周年大庆江泽民总书记在深圳了解了股票市场的情况后回头对一起出席的刘鸿儒说返京途中一起就股市的问题聊一聊在返京的飞机上江泽民总书记在刘鸿儒聊了两个多小时最终江泽民总书记被说服了临下飞机前江泽民对刘鸿儒说股票市场的试点应该保留下来继续试验暂不扩大就这样中国的股票市场算是度过了一次劫难　阅读全文］鹕娇谏系闹飨笔绷鹾枞灞恢扉F基找去谈话刘鸿儒说我这是火山口的工作不好做也做不长朱镕基回答说责任不要你承担我来承担刘鸿儒回答说出了事情哪有让总理承担的当然有我来承担就这样刘鸿儒租了保利大厦的两层楼作为办公室拿着借来的办公经费开始了第一届证监会的工作　阅读全文］＠肴魏蟮摹傲跬贰饼Ｔ诘Ｈ沃ぜ嗷嶂飨的这段时间刘鸿儒获得了同事一个充满情意的绰号刘头年刘头从证监会主席的位置离任开始了他证券人生的另外的辉煌生涯只不过刘鸿儒不再以监管者的身份在哪里摇旗呐喊而是更像一个学者前辈一样呵护中国的资本市场作为一个学者型的官员刘鸿儒一直坚持对证券市场的理论研究与总结　阅读全文］

example 4936 
	sports
	海东青海年月

In [27]:
print("jieba分词，非常费时:\n")
data_words = []
for line in tqdm(content_selected):
    data_words.append([' '.join(jieba.cut(line, cut_all=False))])

  0%|          | 0/96867 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


jieba分词，非常费时:



Loading model cost 0.733 seconds.
Prefix dict has been built succesfully.
100%|██████████| 96867/96867 [03:18<00:00, 488.29it/s]


In [28]:
for i in range(0, 5000, 1234):
    print('sentence %d' % i)
    print(' '.join(data_words[i]))

sentence 0
注 每次 查询 最 多 显示 条
sentence 1234
北京 时间 月 日 欧洲杯 Ｄ 组末 轮 一场 比赛 在 顿涅茨克 顿巴斯 竞技场 展开 争夺 英格兰 客场 比 力克 乌克兰 解禁 复出 的 鲁尼 打入 全场 唯一 入球 英格兰 胜平 不败 战绩 获得 小组 头名 决赛 将 对阵 意大利 ｓ ｐ ｏ ｒ ｔ ｓ 全 体育 图片社 Ｖ 行峦 月 日电 　 据 英国 每日 邮报 报道 财大气粗 的 巴黎 圣日耳曼 队正 密谋 以 万英镑 的 价格 引进 切尔西队 长 约翰 特里 巴黎 圣日耳曼 主帅 安切洛蒂 曾 和 特里 共事 一年 日前 他本 想 以 万英镑 的 天价 引进 Ａ Ｃ 米兰 后卫 蒂 亚戈 席尔瓦 但 这桩 交易 在 最后 一刻 宣布 告吹 豢 ㄋ 尔 财团 注资 后 的 巴黎 圣日耳曼 队 并 不在乎 花钱 他们 有望 以 万英镑 的 价格 引进 那不勒斯 队 射手 拉 维奇 而 为了 得到 特里 他们 准备 承诺 一份 价值 万英镑 周薪 的 天价 待遇
sentence 2468
全景网 月 日 讯 　 钢铁 板块 周 一早 盘 走 弱 Ｓ Ｔ 广钢 跌 逾 ％ 鲁银投资 大 金 重工 等 多只 个股 跌 逾 ％ 因 钢铁行业 经营 状况 继续 恶化 ＞ 荨毒济 参考报 记者 从中 钢协 权威人士 处 获悉 纳入 钢协 统计 的 大中型 钢铁企业 中 月份 盈利 情况 再度 滑至 负数 其中 利润 收入 为 － 亿元 月度 销售 利润率 为 － ％ 全景网 ／ 刘磊
sentence 3702
刘鸿儒 开荒 修路 人 苫 上 拯救 股市 墒 谢挂 不要 搞 　 年 月 深圳 和 珠海 特区 成立 周年 大庆 江泽民 总书记 在 深圳 了解 了 股票市场 的 情况 后 回头 对 一起 出席 的 刘鸿儒 说 返京 途中 一起 就 股市 的 问题 聊一聊 在 返京 的 飞机 上 江泽民 总书记 在 刘鸿儒 聊 了 两个 多 小时 最终 江泽民 总书记 被 说服 了 临下 飞机 前 江泽民 对 刘鸿儒 说 股票市场 的 试点 应该 保留 下来 继续 试验 暂 不 扩大 就 这样 中国 的 股票市场 算是 度过 了 一次 劫难 　 阅读 全文 ］ 鹕 娇谏 系闹 飨 笔 绷 鹾 枞 灞 恢扉 F 基找 去 谈话 刘鸿儒 

In [29]:
# save model
with open('data/sogou_news_test.txt', 'w') as f:
    for i in range(len(data_words)):
        if i%5==0:
            s = '__label__' + label_selected[i] + ' '
            s = s + " ".join([x for x in data_words[i]])
            f.write(s)
            f.write('\n')

with open('data/sogou_news_train.txt', 'w') as f:
    for i in range(len(data_words)):
        if i%5!=0:
            s = '__label__' + label_selected[i] + ' '
            s = s + " ".join([x for x in data_words[i]])
            f.write(s)
            f.write('\n')

In [30]:
import fasttext


In [63]:
lr = 0.05
dim = 256

classifier = fasttext.supervised(input_file = 'data/sogou_news_train.txt',
                                 output = 'data/intent_model',
                                 label_prefix = '__label__',
                                 dim = dim,
                                 lr = lr,
                                 epoch = 5)
result_tr = classifier.test('data/sogou_news_test.txt')


KeyboardInterrupt: 